In [1]:
from openbabel import openbabel

In [2]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [117]:
matches_data = pd.read_csv('./MatchedDataNoStereo/FormoseAmm/NoStereoFormoseAmm.tsv', sep='\t')

In [118]:
FormoseAmm_rels = pd.read_csv('./FormoseAmmData/FormoseAmm_rels.tsv', sep='\t')

In [119]:
def index_finder(Smiles, pathway):
    index = 0
    rule = 0
    for i in range(len(FormoseAmm_rels['Reagent'])):
        if FormoseAmm_rels['Reagent'][i] == Smiles and FormoseAmm_rels['Formed/Produced'][i] == 1 and precursor_finder(FormoseAmm_rels['Index'][i]) not in pathway and precursor_finder(FormoseAmm_rels['Index'][i]) != FormoseAmm_rels['Reagent'][i]:
            index = FormoseAmm_rels['Index'][i] 
            rule = FormoseAmm_rels['Rule'][i]
            break
    return(index, rule)

In [120]:
def precursor_finder(index):
    precursor = 0
    for i in range(len(FormoseAmm_rels['Index'])):
        if FormoseAmm_rels['Index'][i] == index and FormoseAmm_rels['Formed/Produced'][i] == -1:
            precursor = FormoseAmm_rels['Reagent'][i]
            break
    return(precursor)

In [121]:
def pathway_finder(Smiles):
    pathway = [Smiles]
    G0 = ['N', 'C=O', 'C(CO)=O', 'O']
    pathway_complete = False
    while pathway_complete == False:
        index = 0
        rule = 0
        index, rule = index_finder(pathway[-1], pathway)
        if index == 0 or pathway[-1] in G0:
            pathway_complete == True
            break
        else:
            precursor = precursor_finder(index)
            pathway.append([index, rule])
            pathway.append(precursor_finder(index))
    return(pathway)

In [124]:
%%time
pathways = []
checkpoints = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
for i in range(len(matches_data['Smiles'])): 
    pathways.append(pathway_finder(matches_data['Smiles'][i]))
    if i in checkpoints:
        print(i)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
CPU times: user 2h 32min 2s, sys: 29.1 s, total: 2h 32min 31s
Wall time: 2h 33min 55s


In [125]:
G0 = ['N', 'C=O', 'C(CO)=O', 'O']
counter = 0
for i in range(len(pathways)):
    if pathways[i][-1] not in G0:
        counter +=1
print(counter)
print(len(pathways))

0
5106


In [127]:
matches_data['Pathways'] = pathways

In [128]:
matches_data

,Generation,Inchi,Smiles,Pathways
0,G1,YTBSYETUWUMLBZ,C(C(C(CO)O)O)=O,"[C(C(C(CO)O)O)=O, [7_0, Aldol Condensation], C..."
1,G2,RNZJRDIKDNXKIJ,C(C(C(C(CO)O)O)N)=O,"[C(C(C(C(CO)O)O)N)=O, [19_0, Aldol Condensatio..."
2,G2,PYMYPHUHKUWMLA,C(C(C(C(CO)O)O)O)=O,"[C(C(C(C(CO)O)O)O)=O, [27_0, Aldol Condensatio..."
3,G2,JIDVIMQFVJVPPX,C(C(C(CN)O)O)=O,"[C(C(C(CN)O)O)=O, [31_0, Aldol Condensation], ..."
4,G2,UQNPMNUVBRVOCQ,C(C(C(CO)O)N)=O,"[C(C(C(CO)O)N)=O, [33_0, Aldol Condensation], ..."
...,...,...,...,...
5101,G4,ZDOOXLMAGFYHGN,C(CNC(CO)CCO)=O,"[C(CNC(CO)CCO)=O, [164219_0, Strecker Degradat..."
5102,G4,RIVOSRXZPBCYBI,C(CC(CO)NCC=O)(O)=O,"[C(CC(CO)NCC=O)(O)=O, [164223_0, Strecker Degr..."
5103,G4,WTORPHDZWQHEKH,C(CNC(CN)CCO)=O,"[C(CNC(CN)CCO)=O, [164242_0, Strecker Degradat..."
5104,G4,JEPJTNFZHHKCLY,C(CNC(CN)CC(O)=O)=O,"[C(CNC(CN)CC(O)=O)=O, [164257_0, Strecker Degr..."


In [129]:
matches_data.to_csv(r'FormoseAmmPathways.tsv', header=None, index=None, sep='\t', mode='a') 